# Neural search for question answering

In [1]:
!pip install git+https://github.com/apohllo/haystack.git@b79c5b099b294ad5f6cf37a01e4c504b438b8018
# !pip install pymilvus==1.1.2

  Cloning https://github.com/apohllo/haystack.git (to revision b79c5b099b294ad5f6cf37a01e4c504b438b8018) to /private/var/folders/00/3f1t2rhd45l6fvmbkz20ptgh00dr78/T/pip-req-build-s9tfc52j
  Running command git clone -q https://github.com/apohllo/haystack.git /private/var/folders/00/3f1t2rhd45l6fvmbkz20ptgh00dr78/T/pip-req-build-s9tfc52j
^C
ERROR: Operation cancelled by user


In [7]:
!pip install grpcio-tools==1.34.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from haystack.document_stores import FAISSDocumentStore, ElasticsearchDocumentStore
from haystack.nodes import DensePassageRetriever
from haystack.nodes import ElasticsearchRetriever
from tqdm import tqdm
import sys
import os
import regex
import random
import csv
import time
import pandas as pd

/usr/local/lib/python3.8/dist-packages/ray/autoscaler/_private/cli_logger.py:57: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
  warnings.warn(


1. Configure one document store based on ElasticSearch and another document store based on Faiss supported by DPR:
   * The ES store should properly process Polish documents.
   * For DPR you should use enelpol/czywiesz-question and enelpol/czywiesz-context encoders.
   * Warning: Make sure to used models uploaded past 21st of December 2021, since the first model version included a bug.

In [ ]:
!curl -X GET "localhost:9200/"

{
  "name" : "KRKML0004.local",
  "cluster_name" : "elasticsearch",
  "cluster_uuid" : "4RgVB8ahROeKb-55kRVLEQ",
  "version" : {
    "number" : "8.4.3",
    "build_flavor" : "default",
    "build_type" : "tar",
    "build_hash" : "42f05b9372a9a4a470db3b52817899b99a76ee73",
    "build_date" : "2022-10-04T07:17:24.662462378Z",
    "build_snapshot" : false,
    "lucene_version" : "9.3.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


In [ ]:
es_document_store = ElasticsearchDocumentStore(host="localhost", username="", password="", index="polish_index")

In [3]:
faiss_document_store = FAISSDocumentStore(faiss_index_factory_str="Flat")

In [ ]:
es_retriever = ElasticsearchRetriever(es_document_store)

In [4]:
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
Paths used for configuration of notebook: 
    	/root/.jupyter/nbconfig/notebook.json
Paths used for configuration of notebook: 
    	
      - Validating: OK
Paths used for configuration of notebook: 
    	/root/.jupyter/nbconfig/notebook.json


In [5]:
faiss_retriever = DensePassageRetriever(
    document_store=faiss_document_store,
    query_embedding_model="enelpol/czywiesz-question",
    passage_embedding_model="enelpol/czywiesz-context",
)

INFO:haystack.modeling.utils:Using devices: CUDA:0
INFO:haystack.modeling.utils:Number of GPUs: 1


Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/472 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/886k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/543k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/543k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/129 [00:00<?, ?B/s]

INFO:haystack.modeling.model.language_model:LOADING MODEL
INFO:haystack.modeling.model.language_model:=============
INFO:haystack.modeling.model.language_model:Could not find enelpol/czywiesz-context locally.
INFO:haystack.modeling.model.language_model:Looking on Transformers Model Hub (in local cache and online)...


Downloading:   0%|          | 0.00/475M [00:00<?, ?B/s]

INFO:haystack.modeling.model.language_model:Loaded enelpol/czywiesz-context


2. Pre-process all documents from the set of Polish bills (used in the previous exercises), but splitting them into individual articles:
    * You can apply a simple heuristic that searches for Art. at the beginnign of the processed line, to identify the passages.
    * Assing identifiers to the passages by combining the file name with the article id.
    * There might be repeated identifiers, since we use a heuristic. You should ignore that problem - just make sure that you load only one passage with a specific id.

In [6]:
acts = {}

for filename in os.listdir('./ustawy'):
    with open(f'./ustawy/{filename}', 'r', encoding='utf8') as f:
        content = f.read()
        acts[filename] = content

In [7]:
newline_regex = regex.compile(r'\s+')

def format_passage(act):
    result = []
    expected_art_number = 1
    prev_start_pos = None
    for art in regex.compile(r"Art\.\s(\d+)").finditer(act):
        art_number = art.group(1)
        if art_number == str(expected_art_number):
            start_pos = art.start(0)
            if prev_start_pos is not None:
                result.append((expected_art_number - 1, newline_regex.sub(' ', act[prev_start_pos:start_pos])))
            expected_art_number += 1
            prev_start_pos = start_pos
    if prev_start_pos is not None:
        result.append((expected_art_number - 1, newline_regex.sub(' ', act[prev_start_pos:start_pos])))
    return result

# The default format here is:
# {
#    'content': "<DOCUMENT_TEXT_HERE>",
#    'meta': {'name': "<DOCUMENT_NAME_HERE>", ...}
# }

passages = []
for filename, act in acts.items():
    for art, content in format_passage(act):
        passages.append({
            "content": content,
            "meta": {
                 "name": f'{filename[:-4]}_{art}',
            }
        })

In [ ]:
passages[412]

{'content': 'Art. 146. § 1. Przez przyjęcie ładunku odbiorca zobowiązuje się do zapłaty przewoźnikowi jego należności z tytułu frachtu, przestojowego, odszkodowania za przetrzymanie statku i wszelkich innych należności z tytułu przewozu ładunku. § 2. Jeżeli ładunek jest przewożony na podstawie konosamentu, odbiorca obowiązany jest do zapłacenia tylko należności wynikających z konosamentu lub z umowy przewozu, do której postanowień w tym przedmiocie konosament odsyła. § 3. Przy przewozie ładunku na podstawie konosamentu przewoźnik nie może dochodzić od odbiorcy wynagrodzenia za przestój lub odszkodowania za przetrzymanie statku w porcie załadowania, chyba że w konosamencie uwidoczniono czas przestoju lub przetrzymania statku. Jeżeli okres ładowania i wyładowania był określony łącznie jedną liczbą dni lub godzin, przewoźnik nie może wobec odbiorcy powołać się na nadmierną stratę czasu przy ładowaniu, chyba że została ona uwidoczniona w konosamencie. ',
 'meta': {'name': '2001_1545_146'}}

3. Load the passages from previous point to the document stores.

In [ ]:
es_document_store.write_documents(passages)

In [10]:
faiss_document_store.write_documents(passages)
faiss_document_store.update_embeddings(faiss_retriever)

INFO:haystack.document_stores.faiss:Updating embeddings for 21126 docs...


Updating Embedding:   0%|          | 0/21126 [00:00<?, ? docs/s]

/usr/local/lib/python3.8/dist-packages/haystack/modeling/data_handler/dataset.py:65: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  cur_tensor = torch.tensor([sample[t_name] for sample in features], dtype=torch.long)


Create embeddings:   0%|          | 0/10000 [00:00<?, ? Docs/s]

Create embeddings:   0%|          | 0/10000 [00:00<?, ? Docs/s]

Create embeddings:   0%|          | 0/1136 [00:00<?, ? Docs/s]

4. Use the set of questions defined in this dataset to assess the performance of the document stores.

In [11]:
questions = pd.read_csv('./questions.csv', encoding='UTF-8')
questions.head(10)

,passage_id,question,passage,has_answer,year,position,art
0,1997_553_345.txt,"Czy żołnierz, który dopuszcza się czynnej napaści na przełożonego podlega ka...","Art. 345. § 1. Żołnierz, który dopuszcza się czynnej napaści na przełoż...",True,1997,553,345
1,2004_177_21.txt,Z ilu osób składa się komisja przetargowa?,Art. 21. 1. Członków komisji przetargowej powołuje i odwołuje kierownik zam...,True,2004,177,21
2,1996_465_111.txt,Do jakiej wysokości za zobowiązania spółki odpowiada komandytariusz?,Art. 111. Komandytariusz odpowiada za zobowiązania spółki wobec jej wierzy...,True,1996,465,111
3,1994_591_35.txt,"Kiedy ustala się wartość majątku obrotowego, który stracił swoją przydatność?","Art. 35. 1. Wartość rzeczowych składników majątku obrotowego, które utr...",True,1994,591,35
4,2001_1441_74.txt,"Jakiej karze podlega armator, który wykonuje rybołówstwo morskie w polskich...","Art. 74. 1. Armator, który wykonuje rybołówstwo morskie w polskich obszara...",True,2001,1441,74
5,2002_1689_31.txt,Kogo zwalnia się od akcyzy według zasady wzajemności?,"Art. 31. 1. Zwalnia się od akcyzy, jeżeli wynika to z porozumień międzyn...",True,2002,1689,31
6,2001_1353_12.txt,Czy żołnierze przy wykonaniu czynności służbowej nie muszą się przedstawiać?,Art. 12. 1. Żołnierze Żandarmerii Wojskowej przed przystąpieniem do wykon...,True,2001,1353,12
7,1994_592_85.txt,Ile budżetów ma miasto na prawach powiatu?,Art. 85. 1. Miasto na prawach powiatu sporządza jeden budżet. 2. Uchwała b...,True,1994,592,85
8,2001_1353_40.txt,Czy żołnierze Żandarmerii Wojskowej mogą uniemożliwiać ich identyfikację p...,"Art. 40. 1. Żandarmeria Wojskowa, wykonując czynności operacyjno-rozpozna...",True,2001,1353,40
9,1997_735_44.txt,W jakim przypadku policja może dokonać przeszukania pomieszczeń w domu?,Art. 44. §1. W celu znalezienia i zatrzymania przedmiotów podlegających og...,True,1997,735,44


In [12]:
questions_passages = {}

for i, row in questions.iterrows():
    question = row['question']
    passage_id = f"{row['year']}_{row['position']}_{row['art']}"
    if question not in questions_passages:
        questions_passages[question] = []
    questions_passages[question].append(passage_id)
    
# questions_passages

In [ ]:
questions.sample()

,passage_id,question,passage,has_answer,year,position,art
811,2002_1689_125,"Jaki warunek musi być spełniony, aby zabezpieczenie akcyzowe mogło zostać z...","Art. 125. 1. Podatkowe znaki akcyzy otrzymuje zarejestrowany, zgodnie z art....",True,2002,1689,125


In [ ]:
print('ElasticSearch')
for i in range(20): # 20 questions
    question = questions.sample()
    print(question['question'].values[0], question['passage_id'].values[0])
    answer = es_retriever.retrieve(query=question['question'].values[0], top_k=1)
    print(answer[0].content)
    print(answer[0].meta["name"])

ElasticSearch
Jakiej karze podlega osoba, która przywłaszcza sobie cudzą rzecz ruchomą lub prawo majątkowe? 1997_553_284
Art. 284. § 1. Kto przywłaszcza sobie cudzą rzecz ruchomą lub prawo majątkowe, podlega karze pozbawienia wolności do lat 3. § 2. Kto przywłaszcza sobie powierzoną mu rzecz ruchomą, podlega karze pozbawienia wolności od 3 miesięcy do lat 5. § 3. W wypadku mniejszej wagi lub przywłaszczenia rzeczy znalezionej, sprawca podlega grzywnie, karze ograniczenia wolności albo pozbawienia wolności do roku. § 4. Jeżeli przywłaszczenie nastąpiło na szkodę osoby najbliższej, ściganie następuje na wniosek pokrzywdzonego. 
1997_553_284
Czy zakłady zachowawcze zaliczane są do grupy działalności charytatywnych jakie mogą prowadzić gminy żydowskie? 1997_251.txt-Art. 13.
Art. 18. Gminy żydowskie oraz inne osoby prawne, działające na podstawie ustawy, mogą prowadzić działalność charytatywną, a w szczególności zakłady wychowawcze, opiekuńcze i opieki zdrowotnej. 
1997_251_18
Czy żółnie

In [13]:
print('FAISS')
for i in range(20): # 20 questions
    question = questions.sample()
    print(question['question'].values[0], question['passage_id'].values[0])
    answer = faiss_retriever.retrieve(query=question['question'].values[0])
    print(answer[0].content)
    print(answer[0].meta["name"])

FAISS
Kto wyznacza termin usunięcia braków wynikłych z niedopełnienia przepisów prawa przez spółkę? 1996_465_17
Art. 21. §1. Sąd rejestrowy może orzec o rozwiązaniu wpisanej do rejestru spółki kapitałowej w przypadku, gdy: 1) nie zawarto umowy spółki, 2) określony w umowie albo statucie przedmiot działalności spółki jest sprzeczny z prawem, 3) umowa albo statut spółki nie zawiera postanowień dotyczących firmy, przedmiotu działalności spółki, kapitału zakładowego lub wkładów, 4) wszystkie osoby zawierające umowę spółki albo podpisujące statut nie miały zdolności do czynności prawnych w chwili ich dokonywania. §2. W przypadkach określonych w § 1, jeżeli braki nie zostaną usunięte w terminie wyznaczonym przez sąd rejestrowy, sąd ten może, po wezwaniu zarządu spółki do złożenia oświadczenia, wydać postanowienie o rozwiązaniu spółki. §3. Jeżeli braki, o których mowa w § 1, nie mogą być usunięte, sąd rejestrowy orzeka o rozwiązaniu spółki. §4. Z powodu braków, o których mowa w § 1, spółka ni

5. Compare the performance of the data stores using the following metrics: Pr@1, Rc@1, Pr@3, Rc@3.

In [14]:
def get_metrics(model, n, row):
    start = time.time()
    answers = model.retrieve(query=row['question'], top_k=n)
    end = time.time()
    i = 0
    for answer in answers:
        passage_id = answer.meta["name"]
        if passage_id in questions_passages[row['question']]:
            i += 1
   
    return (1.0*i/n, 1.0*i/(i+n), end - start)

def calculate_metrics(model):
    data = []
    questions_list = questions['question'].values
    for index, row in questions.iterrows():
        precision_1, recall_1, time_1 = get_metrics(model, 1, row)
        precision_3, recall_3, time_3 = get_metrics(model, 3, row)
        data.append((precision_1, recall_1, time_1, precision_3, recall_3, time_3))
    columns = ['Pr@1', 'Rc@1', 'Time@1', 'Pr@3', 'Rc@3', 'Time@3']
    return pd.DataFrame(data=data, columns=columns)

In [ ]:
es_metrics = calculate_metrics(es_retriever)
print("ElastiSearch metrics")
print(f'Pr@1: {es_metrics["Pr@1"].mean()}')
print(f'Rc@1: {es_metrics["Rc@1"].mean()}')
print(f'Time@1: {es_metrics["Time@1"].mean()}')
print(f'Pr@3: {es_metrics["Pr@3"].mean()}')
print(f'Rc@3: {es_metrics["Rc@3"].mean()}')
print(f'Time@3: {es_metrics["Time@3"].mean()}')

ElastiSearch metrics
Pr@1: 0.7674576271186441
Rc@1: 0.38372881355932204
Time@1: 0.0045271039413193525
Pr@3: 0.5012429378531073
Rc@3: 0.30311864406779665
Time@3: 0.00483429650128898


In [15]:
faiss_metrics = calculate_metrics(faiss_retriever)
print("FAISS metrics")
print(f'Pr@1: {faiss_metrics["Pr@1"].mean()}')
print(f'Rc@1: {faiss_metrics["Rc@1"].mean()}')
print(f'Time@1: {faiss_metrics["Time@1"].mean()}')
print(f'Pr@3: {faiss_metrics["Pr@3"].mean()}')
print(f'Rc@3: {faiss_metrics["Rc@3"].mean()}')
print(f'Time@3: {faiss_metrics["Time@3"].mean()}')

FAISS metrics
Pr@1: 0.4101694915254237
Rc@1: 0.20508474576271185
Time@1: 0.018098753266415354
Pr@3: 0.3157062146892656
Rc@3: 0.19989830508474576
Time@3: 0.018526975179122666


6. Answer the following questions:
    * Which of the document stores performs better? Take into account the different metrics enumerated in the previous point.
    
        We can observe that ElasticSearch has superior performance in comparison with FAISS.
    
    * Which of the document stores is faster?
    
        Saving as well as retrieving data is faster with ElasticSearch.
    
    * Try to determine the other pros and cons of using sparse and dense document retrieval models.
    
        One of the limitations of sparse retrieval is that, since it represents each dialogue context and response using the existing terms in a bag-of-words manner, the vocabulary mismatch problem might occur.
        Dense retrieval needs large datasets in order to beat a strong sparse retrieval baseline in the zero-shot setting. The closer the intermediate training data distribution is to the target domain, the better the dense retrieval model performs.

